In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
#!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-100k.zip
#!unzip ml-1m.zip

--2020-02-15 18:50:22--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  6.48MB/s    in 0.9s    

2020-02-15 18:50:23 (6.48 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [0]:
import pandas as pd
import torch as torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.tensorboard import SummaryWriter
#from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px
import math
from torch.utils.data import Dataset

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [0]:
# Read and process movie metadata
movies=pd.read_csv('/content/ml-100k/u.item',sep='|',header=None,encoding='latin-1',
                   names=['movieId', 'title', 'date', 'N', 'url', *[f'g{i}' for i in range(19)]])
movies['movieId_codes'] = movies['movieId'].astype('category').cat.codes

In [0]:
ratings=pd.read_csv('/content/ml-100k/u.data',sep='\t',header=None,names=['userId','movieId','rating','timestamp'])
ratings=ratings.join(movies.set_index('movieId'),on='movieId')

In [0]:
# Read and process users metadata
users = pd.read_csv('/content/ml-100k/u.user',sep='|',header=None, names=['userId', 'age' ,'gender','occupation', 'zipcode'])
users['age_bins'] = pd.cut(x=users['age'], bins=[0,15,20, 30, 40, 50,100])
users['gender_codes'] = users['gender'].astype('category').cat.codes
users['age_codes'] = users['age_bins'].astype('category').cat.codes
users['occupation_codes'] = users['occupation'].astype('category').cat.codes
users['userId_codes'] = users['userId'].astype('category').cat.codes

In [0]:
ratings=ratings.join(users.set_index('userId'), on='userId')

In [0]:
ratings

,userId,movieId,rating,timestamp,title,date,N,url,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,movieId_codes,age,gender,occupation,zipcode,age_bins,gender_codes,age_codes,occupation_codes,userId_codes
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,241,49,M,writer,55105,"(40, 50]",1,4,20,195
1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,301,39,F,executive,00000,"(30, 40]",0,3,6,185
2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Heavyweights%...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,376,25,M,writer,40206,"(20, 30]",1,2,20,21
3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Legends%20of%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,50,28,M,technician,80525,"(20, 30]",1,2,19,243
4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-11...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,345,47,M,educator,55113,"(40, 50]",1,4,3,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,880175444,"First Wives Club, The (1996)",14-Sep-1996,NaN,http://us.imdb.com/M/title-exact?First%20Wives...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,475,13,M,student,83702,"(0, 15]",1,0,18,879
99996,716,204,5,879795543,Back to the Future (1985),01-Jan-1985,NaN,http://us.imdb.com/M/title-exact?Back%20to%20t...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,203,36,F,administrator,44265,"(30, 40]",0,3,0,715
99997,276,1090,1,874795795,Sliver (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Sliver%20(1993),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1089,21,M,student,95064,"(20, 30]",1,2,18,275
99998,13,225,2,882399156,101 Dalmatians (1996),27-Nov-1996,NaN,http://us.imdb.com/M/title-exact?101%20Dalmati...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,224,47,M,educator,29206,"(40, 50]",1,4,3,12


In [0]:
feature_columns = ['userId_codes','movieId_codes',*[f'g{i}' for i in range(19)],'age_codes','gender_codes','occupation_codes']
features_width = {
    'userId_codes':len(ratings['userId_codes'].unique()),
    'movieId_codes':len(ratings['movieId_codes'].unique()),
    **{f'g{i}':1 for i in range(19)},
    'age_codes':len(ratings['age_codes'].unique()),
    'gender_codes':len(ratings['gender_codes'].unique()),
    'occupation_codes':len(ratings['occupation_codes'].unique()),
}

In [0]:
ratings[feature_columns]

,userId_codes,movieId_codes,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,age_codes,gender_codes,occupation_codes
0,195,241,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,20
1,185,301,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,3,0,6
2,21,376,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,20
3,243,50,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,2,1,19
4,165,345,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,879,475,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,18
99996,715,203,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,0
99997,275,1089,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1,18
99998,12,224,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,3


In [0]:
# calculate offsets
#start from 1, 0 is padding index
next_offset = 1
features_offsets={}
for k,v in features_width.items():
  features_offsets[k] = next_offset
  next_offset += v

In [0]:
# map all column indides to start from correct offset
for categorical_column in ['userId_codes','movieId_codes','age_codes','gender_codes','occupation_codes']:
  ratings[categorical_column] = ratings[categorical_column].apply(lambda c: c + features_offsets[categorical_column])

for indicator_column in [f'g{i}' for i in range(19)]:
  ratings.loc[:,indicator_column] *= features_offsets[indicator_column]

In [0]:
ratings[feature_columns]

,userId_codes,movieId_codes,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,age_codes,gender_codes,occupation_codes
0,196,1185,0,0,0,0,0,2631,0,0,0,0,0,0,0,0,0,0,0,0,0,2649,2652,2673
1,186,1245,0,0,0,0,0,0,2632,0,0,0,2636,0,0,2639,0,0,2642,0,0,2648,2651,2659
2,22,1320,0,0,0,0,2630,2631,0,0,0,0,0,0,0,0,0,0,0,0,0,2647,2652,2673
3,244,994,0,0,0,0,0,0,0,0,2634,0,0,0,0,0,2640,0,0,2643,2644,2647,2652,2672
4,166,1289,0,0,0,0,0,0,2632,0,2634,0,0,0,0,0,0,0,0,0,0,2649,2652,2656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,1419,0,0,0,0,0,2631,0,0,0,0,0,0,0,0,0,0,0,0,0,2645,2652,2671
99996,716,1147,0,0,0,0,0,2631,0,0,0,0,0,0,0,0,0,2641,0,0,0,2648,2651,2653
99997,276,2033,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2642,0,0,2647,2652,2671
99998,13,1168,0,0,0,0,2630,2631,0,0,0,0,0,0,0,0,0,0,0,0,0,2649,2652,2656


In [0]:
# create mapping from 
#dict( zip( movies['movieId'].cat.codes, df['title'] ) )

In [0]:
# This will bring us back to matrix factorization:
#X = torch.tensor(ratings[['userId_codes','movieId_codes']].values)
#data_x = X

In [0]:
# TODO yonigo - minimize matrix width to biggest row
X = torch.tensor(ratings[feature_columns].values)
data_x = torch.zeros_like(X)
for i in range(X.shape[0]):
  r = X[i,:]
  non_zeros = r[r!=0]
  data_x[i,0:(non_zeros.shape[0])] = non_zeros

In [0]:
data_y = torch.tensor(ratings['rating'].values).float()
dataset = data.TensorDataset(data_x, data_y)

In [0]:
bs=128
train_n = int(len(dataset)*0.9)
valid_n = len(dataset) - train_n
splits = [train_n,valid_n]
assert sum(splits) == len(dataset)
trainset,devset = torch.utils.data.random_split(dataset,splits)
train_dataloader = data.DataLoader(trainset,batch_size=bs,shuffle=True)
dev_dataloader = data.DataLoader(devset,batch_size=bs,shuffle=True)

In [0]:
# copied from fastai: 
def trunc_normal_(x, mean=0., std=1.):
    "Truncated normal initialization."
    # From https://discuss.pytorch.org/t/implementing-truncated-normal-initializer/4778/12
    return x.normal_().fmod_(2).mul_(std).add_(mean)

In [0]:
class NNet(nn.Module):
  def __init__(self, n, k):
    super().__init__()

    self.w0 = nn.Parameter(torch.zeros(1))
    self.bias = nn.Embedding(n, 1)
    self.embeddings = nn.Embedding(n, k, padding_idx=0) # this is V matrix from paper

    # See https://arxiv.org/abs/1711.09160
    with torch.no_grad(): trunc_normal_(self.embeddings.weight, std=0.01)
    with torch.no_grad(): trunc_normal_(self.bias.weight, std=0.01)

  def forward(self, X):
    emb = self.embeddings(X)

    # zero the embeddings of padding indices
    mask = X != 0
    emb = emb*(mask.unsqueeze(2))

    # calculate the interactions in complexity of O(nk) see lemma 3.1 from paper
    a = emb.sum(dim=1).pow(2)
    b = emb.pow(2).sum(dim=1)
    pairwise = (a-b).sum(1)*0.5
    bias = self.bias(X).squeeze().sum(1)
    return torch.sigmoid(self.w0 + bias + pairwise)*5.5


In [0]:
# fit/test functions
def fit(iterator, model, optimizer, criterion):
    train_loss = 0
    model.train()
    for x,y in iterator:
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        train_loss += loss.item()*x.shape[0]
        loss.backward()
        optimizer.step()
    return train_loss / len(iterator.dataset)

def test(iterator, model, criterion):
    train_loss = 0
    model.eval()
    for x,y in iterator:                    
        with torch.no_grad():
            y_hat = model(x)
        loss = criterion(y_hat, y)
        train_loss += loss.item()*x.shape[0]
    return train_loss / len(iterator.dataset)

In [0]:
def train_n_epochs(model, n, optimizer):

  criterion = nn.MSELoss().to(device)
  for epoch in range(n):
    start_time = time.time()
    train_loss = fit(train_dataloader, model, optimizer, criterion)
    valid_loss = test(dev_dataloader, model, criterion)
    secs = int(time.time() - start_time)
    print(f'Epoch: {epoch} time: {secs}[s]')
    print(f'\tTrain Loss: {(train_loss):.4f}')
    print(f'\tValidation Loss: {(valid_loss):.4f}')

In [0]:
features = data_x.max() + 1
model = NNet(features,50)
wd=1e-5
lr=0.001
epochs=6
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
train_n_epochs(model,epochs,optimizer)

Epoch: 0 time: 9[s]
	Train Loss: 0.9723
	Validation Loss: 0.9036
Epoch: 1 time: 8[s]
	Train Loss: 0.8672
	Validation Loss: 0.8852
Epoch: 2 time: 9[s]
	Train Loss: 0.8285
	Validation Loss: 0.8723
Epoch: 3 time: 9[s]
	Train Loss: 0.7857
	Validation Loss: 0.8571
Epoch: 4 time: 8[s]
	Train Loss: 0.7427
	Validation Loss: 0.8513
Epoch: 5 time: 8[s]
	Train Loss: 0.6999
	Validation Loss: 0.8431


In [0]:
# now lets see what genres are best to recommend to a woman
woman_code = 0
woman_offset = woman_code + features_offsets['gender_codes']
woman_embeddings = model.embeddings(torch.tensor(woman_offset))
#woman_bias = model.bias(torch.tensor(woman_offset))

In [0]:
# embeddings of genres:
genre_embeddings = model.embeddings(torch.tensor([features_offsets[i] for i in [f'g{i}' for i in range(19)]]))
genre_bias = model.bias(torch.tensor([features_offsets[i] for i in [f'g{i}' for i in range(19)]]))

In [0]:
(genre_bias.squeeze()+(woman_embeddings*genre_embeddings).sum(1))

tensor([ 0.0251, -0.0744, -0.0320,  0.0354, -0.0838,  0.0144, -0.0313,  0.1628,
         0.1836, -0.0425,  0.0563, -0.1282,  0.0401,  0.0204,  0.0976, -0.0521,
         0.0238,  0.0788,  0.0516], grad_fn=<AddBackward0>)

In [0]:
a.unsqueeze(1).shape

torch.Size([3, 1])

In [0]:
a.T.shape

torch.Size([3])